## **IMPORTS**

In [3]:
import pandas as pd

### **DATA CLEANING**

In [4]:
df_historical_data = pd.read_csv('fifa_worldcup_historical_data.csv')
df_fixtures = pd.read_csv('fifa_worldcup_2022_fixtures.csv')
df_missing_data = pd.read_csv('fifa_worldcup_missing_data.csv')

In [5]:
# Clean whitespace from team names in fixtures data
df_fixtures["home"] = df_fixtures["home"].str.strip()
df_fixtures["away"] = df_fixtures["away"].str.strip()

In [6]:
df_missing_data.isnull().sum()
df_historical_data.isnull().sum()
df_fixtures.isnull().sum()

home     0
score    0
away     0
year     0
dtype: int64

- we see that there is no null data in our dataframes

In [7]:
# concat historical data with missing data
df_historical_results_data = pd.concat([df_historical_data, df_missing_data], ignore_index=True)
df_historical_results_data.drop_duplicates(inplace=True)
df_historical_results_data.sort_values(by='year', inplace=True)
df_historical_results_data

,home,score,away,year
0,France,4–1,Mexico,1930
17,Uruguay,4–2,Argentina,1930
16,Uruguay,6–1,Yugoslavia,1930
15,Argentina,6–1,United States,1930
14,Paraguay,1–0,Belgium,1930
...,...,...,...,...
828,Serbia,0–2,Brazil,2018
827,Serbia,1–2,Switzerland,2018
826,Brazil,2–0,Costa Rica,2018
824,Costa Rica,0–1,Serbia,2018


In [8]:
# we want to delete a walk over match where one team did not show up so that our model is not affected by it
delete_id = df_historical_results_data[df_historical_results_data['home'].str.contains("Sweden") & df_historical_results_data['away'].str.contains("Austria")].index
df_historical_results_data.drop(delete_id , inplace=True)

In [9]:
df_historical_results_data["score"].str.contains("[^\d-]")

0      True
17     True
16     True
15     True
14     True
       ... 
828    True
827    True
826    True
824    True
811    True
Name: score, Length: 899, dtype: bool

In [10]:
# Clean whitespace from team names in fixtures data
df_historical_results_data["home"] = df_historical_results_data["home"].str.strip()
df_historical_results_data["away"] = df_historical_results_data["away"].str.strip()

In [11]:
# split score column into home_score and away_score for better analysis, also drop the original score column
df_historical_results_data[['home_goals', 'away_goals']] = df_historical_results_data['score'].str.split('–', expand=True)
df_historical_results_data.drop('score', axis=1, inplace=True)

In [12]:
df_historical_results_data['home_goals'] = df_historical_results_data['home_goals'].str.replace('[^\d–]', '', regex=True)
df_historical_results_data['away_goals'] = df_historical_results_data['away_goals'].str.replace('[^\d–]', '', regex=True)

In [13]:
# Rename columns for clarity
df_historical_results_data.rename(columns={'home': 'home_team', 'away': 'away_team'}, inplace=True)

In [14]:
df_historical_results_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 899 entries, 0 to 811
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   home_team   899 non-null    object
 1   away_team   899 non-null    object
 2   year        899 non-null    int64 
 3   home_goals  899 non-null    object
 4   away_goals  899 non-null    object
dtypes: int64(1), object(4)
memory usage: 42.1+ KB


In [15]:
# convert home_goals and away_goals to numeric
df_historical_results_data['home_goals'] = pd.to_numeric(df_historical_results_data['home_goals'])
df_historical_results_data['away_goals'] = pd.to_numeric(df_historical_results_data['away_goals'])
df_historical_results_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 899 entries, 0 to 811
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   home_team   899 non-null    object
 1   away_team   899 non-null    object
 2   year        899 non-null    int64 
 3   home_goals  899 non-null    int64 
 4   away_goals  899 non-null    int64 
dtypes: int64(3), object(2)
memory usage: 42.1+ KB


In [16]:
# adding total game goals column
df_historical_results_data['total_goals'] = df_historical_results_data['home_goals'] + df_historical_results_data['away_goals']
df_historical_results_data.head()

,home_team,away_team,year,home_goals,away_goals,total_goals
0,France,Mexico,1930,4,1,5
17,Uruguay,Argentina,1930,4,2,6
16,Uruguay,Yugoslavia,1930,6,1,7
15,Argentina,United States,1930,6,1,7
14,Paraguay,Belgium,1930,1,0,1


### **SAVE CLEAN DATAFRAMES AS CSV FILES**

In [18]:
df_historical_results_data.to_csv('fifa_worldcup_historical_results_data_cleaned.csv', index=False)
df_fixtures.to_csv('fifa_worldcup_2022_fixtures_cleaned.csv', index=False)